In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [11]:
# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [17]:
# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()
print(type(documents[0]),documents[0].page_content[:10])

<class 'langchain_core.documents.base.Document'> 제12조(비과세소득


In [13]:

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
#print(split_docs)

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
#print(retrieved_docs)

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용한 결과
('소득세법 제12조에 따르면, 다음과 같은 소득은 비과세 소득으로 간주되어 소득세를 부과하지 않습니다:\n'
 '\n'
 '1. 「공익신탁법」에 따른 공익신탁의 이익\n'
 '2. 사업소득 중 특정한 조건을 충족하는 소득\n'
 '   - 논ㆍ밭을 작물 생산에 이용해서 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외)\n'
 '   - 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득 (2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목 벌채 또는 양도로 발생한 소득 중 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '\n'
 '이러한 소득들은 특정한 조건을 만족해야 비과세 혜택을 받을 수 있으며, 관련 세부사항은 대통령령으로 규정됩니다.')
context 적용하지 않은 결과
('소득세법에서 비과세소득은 과세 대상이 되지 않는 소득을 말합니다. 이는 특정한 소득에 대해 세금을 부과하지 않도록 법으로 정해진 '
 '것입니다. 비과세소득의 종류는 각 국가마다 다를 수 있으나, 일반적으로 다음과 같은 것들이 포함될 수 있습니다.\n'
 '\n'
 '1. **복지 관련 소득**: 사회복지 혜택이나 정부로부터 제공되는 보조금 등은 종종 비과세로 처리됩니다.\n'
 '\n'
 '2. **장학금**: 학생이 교육을 위한 목적으로 받은 장학금이나 교육 관련 지원금은 대부분 비과세로 처리됩니다.\n'
 '\n'
 '3. **보험금**: 생명보험금 또는 특정 종류의 보험금 등이 비과세 소득으로 인정될 수 있습니다.\n'
 '\n'
 '4. **임직원의 복지혜택**: 직원들이 받는 특정 복지 혜택, 